# Agente Consultor de Microsoft Learn vía MCP Remoto

## Descripción

En este ejercicio, desarrollarás un **agente especializado en ingeniería aeroespacial** que utiliza el protocolo MCP (Model Context Protocol) remoto para acceder a documentación técnica de Microsoft Learn y Azure.

### Arquitectura MCP Remoto
```
Pregunta técnica
    ↓
[AeroDocsAgent]
    ↓
[MCPStreamableHTTPTool]
    ↓
https://learn.microsoft.com/api/mcp
    ↓
Respuesta basada en documentación Microsoft
```

### Cargar librerías y variables de entorno

In [ ]:
import asyncio
import os
from dotenv import load_dotenv, find_dotenv
from agent_framework import ChatAgent, MCPStreamableHTTPTool
from agent_framework.azure import AzureAIAgentClient
from azure.identity.aio import AzureCliCredential

load_dotenv(find_dotenv(usecwd=True))

### Crear y ejecutar el agente con herramienta MCP remota

Configuramos el `ChatAgent` con instrucciones para usar exclusivamente el servidor MCP de Microsoft Learn. La herramienta `MCPStreamableHTTPTool` se conecta a la API remota.

In [ ]:
async def query_aero_docs(prompt):
    print("Connecting to remote MCP server...")
    async with (
        AzureCliCredential() as credential,
        MCPStreamableHTTPTool(
            name="mslearn",
            url="https://learn.microsoft.com/api/mcp",
            headers={"Authorization": "Bearer your-token"},  # Example token
        ) as mcp_server,
        ChatAgent(
            chat_client=AzureAIAgentClient(async_credential=credential),
            name="AeroDocsAgent",
            instructions=(
                """
                Eres un agente de IA especializado en ingeniería aeroespacial y fabricación aeronáutica.
                Limita tus respuestas EXCLUSIVAMENTE a la documentación de Microsoft Learn (Microsoft/Azure)
                accesible a través del servidor MCP de Microsoft Learn.
                Explica los conceptos con precisión, proporciona fórmulas cuando apliquen y sugiere pasos prácticos.
                Si la pregunta es ambigua, solicita aclaraciones breves. Responde en español por defecto.
                """
            ),
        ) as agent,
    ):
        print(f"Running query: {prompt}")
        result = await agent.run(prompt, tools=mcp_server)
        print("Query result:")
        print(result)


In [ ]:
# Enter your question here
print("Starting MCP remote query...")
user_prompt = "Cómo configurar Azure AI Search para indexar documentación técnica (PDF) y usarlo con Azure OpenAI en un patrón RAG para consultas internas de ingeniería."
await query_aero_docs(user_prompt)

In [ ]:
# Enter your question here
print("Starting MCP remote query...")
user_prompt = "Guía paso a paso para crear un workspace de Azure Machine Learning, entrenar un modelo de mantenimiento predictivo con datos de sensores y desplegar un endpoint en línea."
await query_aero_docs(user_prompt)